In [112]:
import pandas 

In [113]:
dataframeResults = pandas.read_csv("subdataset/results.csv")
dataframeDrivers = pandas.read_csv("subdataset/drivers.csv")
dataframeConstructors = pandas.read_csv("subdataset/constructors.csv")
dataframeStatus = pandas.read_csv("subdataset/status.csv")
dataframeRaces = pandas.read_csv("subdataset/races.csv")

In [114]:
dataframeResults = dataframeResults.merge(dataframeDrivers, on = "driverId", how = "left")
dataframeResults = dataframeResults.merge(dataframeConstructors, on = "constructorId", how = "left")
dataframeResults = dataframeResults.merge(dataframeStatus, on = "statusId", how = "left")
dataframeResults = dataframeResults.merge(dataframeRaces, on = "raceId", how="left")
dataframeResults = dataframeResults.drop(columns=["driverId","constructorId",'url_x','nationality_x','url_y','nationality_y','statusId'])
dataframeResults = dataframeResults.drop(columns=["dob","positionText","positionOrder","number_x","number_y"])
dataframeResults = dataframeResults.drop(columns=["fp1_date","fp1_time","fp2_date","fp2_time","fp3_date","fp3_time"])
dataframeResults = dataframeResults.drop(columns=["quali_date","quali_time","sprint_date","sprint_time","url","time_y","date"])
dataframeResults = dataframeResults.drop(columns=["resultId","code","circuitId","forename","surname"])
dataframeResults = dataframeResults.drop(columns=["rank","fastestLap","round","time_x","raceId","name_x"])
dataframeResults = dataframeResults.loc[:,["year","name_y","driverRef","constructorRef","grid","position","points","status","fastestLapTime","fastestLapSpeed","laps","milliseconds"]]
dataframeResults = dataframeResults.replace("\\N","NaN")

In [116]:
dataframeResults.to_csv("dataset.csv")